<a href="https://colab.research.google.com/github/MaheepChaudhary/GANs_mnist/blob/main/GANs_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing the necessary packages
import tensorflow as tf
import tensorflow.keras as tk
from tensorflow.keras.datasets import mnist 
import imageio

In [ ]:
#preprocessing of dataset(normalization and shuffling of images)


In [ ]:
#generator model


In [ ]:
#discriminator model


In [ ]:
#loss


In [ ]:
#optimizers


In [ ]:
#generative_loss


In [ ]:
#discriminator_loss


In [ ]:
#optimizer


In [ ]:
#training_Step


In [ ]:
#training_function


In [17]:
#plotting and saving image function


In [ ]:
#training occurs
